In [61]:
import tensorflow as tf
from numpy.random import RandomState
#定义训练数据batch的大小
batch_size=8

#定义神经网络的参数，这里还是沿用3
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,3],stddev=1,seed=1))
w3=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))


#placeholder是TensorFlow的占位符节点，shape选填
x=tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

a=tf.matmul(x,w1)
b=tf.matmul(a,w2)
y=tf.matmul(b,w3)

y=tf.sigmoid(y)
#熵越大，说明不确定性越大
cross_entropy= -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,-1,1.0))+
    (1 - y_)*tf.log(tf.clip_by_value(1-y,-1,1.0)))
#tf.clip_by_value(A, min, max)：输入一个张量A，把A中的每一个元素的值都压缩在min和max之间。
#小于min的让它等于min，大于max的元素的值等于max
train_step=tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

#通过随机数生成一个模拟数据集
rdm=RandomState(1)
dataset_size=128
X=rdm.rand(dataset_size,2)
'''定义规则出来的样本标签。在这里所有x1+x2<1的样例被认为是正样本'''
Y=[[int(x1+x2<1)] for (x1,x2) in X]

with tf.Session() as sess:
    #初始化变量
    init_op=tf.global_variables_initializer()
    sess.run(init_op)
    
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS=50000
    
    for i in range(STEPS):
        #每次选取batch_size个样本进行训练
        start=(i*batch_size)%dataset_size#从128个样本中选取
        end=min(start+batch_size,dataset_size)#规模为最大为8的数据集

        #通过选取的样本训练神经网络并更新参数
        sess.run(train_step,
                 feed_dict={x:X[start:end],y_:Y[start:end]})
    
        if(i%1000==0):
            #每隔一段时间计算在所有数据上的交叉熵并输出
            total_cross_entropy=sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training step,cross entropy on all data is %g"%(i,total_cross_entropy))

    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]
 [ 0.59282297 -2.1229296  -0.72289723]]
After 0 training step,cross entropy on all data is 1.56615
After 1000 training step,cross entropy on all data is 0.608266
After 2000 training step,cross entropy on all data is 0.608132
After 3000 training step,cross entropy on all data is 0.608198
After 4000 training step,cross entropy on all data is 0.608113
After 5000 training step,cross entropy on all data is 0.608146
After 6000 training step,cross entropy on all data is 0.608103
After 7000 training step,cross entropy on all data is 0.608115
After 8000 training step,cross entropy on all data is 0.608099
After 9000 training step,cross entropy on all data is 0.608105
After 10000 training step,cross entropy on all data is 0.608099
After 11000 training step,cross entropy on all data is 0.608103
After 12000 training step,cross entropy on all da